In [303]:
from dotenv import load_dotenv
import os
import google.generativeai as genai
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
import sqlite3
from sqlalchemy import create_engine, MetaData
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq


In [304]:
load_dotenv()

True

In [305]:
template = """
Based on the table schema below, write a SQL query that would answer the user's question. Return only the query with semicolon at the end without newline characters.:
{schema}
Write only the SQL query and nothing else. Do not wrap the SQL query in any other text, not even backticks. Remove any special characters such as / or \\
    
    For example:
    Question: which 3 artists have the most tracks?
    SQL Query: SELECT ArtistId, COUNT(*) as track_count FROM Track GROUP BY ArtistId ORDER BY track_count DESC LIMIT 3;
    Question: Name 10 artists
    SQL Query: SELECT Name FROM Artist LIMIT 10;
Question: {question}
SQL Query: 
"""

prompt = ChatPromptTemplate.from_template(template)

In [306]:
prompt.format(schema="my_schema", question="how many users are there?")

"Human: \nBased on the table schema below, write a SQL query that would answer the user's question. Return only the query with semicolon at the end without newline characters.:\nmy_schema\nWrite only the SQL query and nothing else. Do not wrap the SQL query in any other text, not even backticks. Remove any special characters such as / or \\\n    \n    For example:\n    Question: which 3 artists have the most tracks?\n    SQL Query: SELECT ArtistId, COUNT(*) as track_count FROM Track GROUP BY ArtistId ORDER BY track_count DESC LIMIT 3;\n    Question: Name 10 artists\n    SQL Query: SELECT Name FROM Artist LIMIT 10;\nQuestion: how many users are there?\nSQL Query: \n"

In [307]:
db_path = os.path.join('../', 'mydatabase.db')
db_uri = f"sqlite:///{db_path}"


db = SQLDatabase.from_uri(db_uri)

def get_schema(_):
    return db.get_table_info()


In [308]:
# os.environ["GOOGLE_API_KEY"]  = os.getenv("GEMINI_API_KEY")

# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash"
# )
# llm


os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0)


In [309]:

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm
    | StrOutputParser()
)

sql_chain

RunnableAssign(mapper={
  schema: RunnableLambda(get_schema)
})
| ChatPromptTemplate(input_variables=['question', 'schema'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'schema'], template="\nBased on the table schema below, write a SQL query that would answer the user's question. Return only the query with semicolon at the end without newline characters.:\n{schema}\nWrite only the SQL query and nothing else. Do not wrap the SQL query in any other text, not even backticks. Remove any special characters such as / or \\\n    \n    For example:\n    Question: which 3 artists have the most tracks?\n    SQL Query: SELECT ArtistId, COUNT(*) as track_count FROM Track GROUP BY ArtistId ORDER BY track_count DESC LIMIT 3;\n    Question: Name 10 artists\n    SQL Query: SELECT Name FROM Artist LIMIT 10;\nQuestion: {question}\nSQL Query: \n"))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x11f84af80>, async_client=<groq.resou

In [310]:
sql_chain.invoke({"question":"how many albums records are there?"})

'SELECT COUNT(*) FROM Album;'

In [311]:
template = """
Based on the table schema below, question, sql_query, and sql_response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}
"""
prompt = ChatPromptTemplate.from_template(template)

In [312]:
def run_query(query):
    return db.run(query)

In [313]:
# run_query('SELECT COUNT(*) FROM artist;\n')



run_query(sql_chain.invoke({"question":"how many albums records are there?"}))

'[(347,)]'

In [314]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response = lambda variables: run_query(variables["query"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [315]:
full_chain.invoke({"question":"how many albums are there?"})

OperationalError: (sqlite3.OperationalError) unrecognized token: "\"
[SQL: SELECT COUNT(\*) FROM Album;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)